# Growth requirements

This notebook is focused on carbon sources, minimal media, uptakes etc. of the models present in this repository.

*Author: Famke Baeuerle*

In [22]:
import refinegems as rg
from cobra.medium import minimal_medium
import pandas as pd

CARBON = ["EX_gly_e", "EX_fum_e", "EX_male_e", "EX_pyr_e", "EX_succ_e", "EX_glc__D_e", "EX_urea_e", "EX_22bipy_e", "EX_boricacid_e", 
            "EX_adocbl_e", "EX_4abz_e", "EX_pnto__R_e", "EX_pydam_e", "EX_nac_e", 'EX_fru_e', 'EX_succ_e', 'EX_gam_e', 'EX_glyald_e']
NITROGEN = ["EX_ala__L_e", "EX_arg__L_e", "EX_cys__L_e", "EX_glu__L_e", "EX_his__L_e", "EX_leu__L_e", "EX_lys__L_e", "EX_orn_e", "EX_phe__L_e", 
            "EX_pro__L_e", "EX_ser__L_e", "EX_thr__L_e", "EX_trp__L_e", "EX_val__L_e", "EX_cit_e", "EX_btn_e", "EX_ribflv_e", "EX_thm_e"]

## Carbon sources

In [46]:
def simulate_carbon_sources(model, base_medium):
    carbon_sources = CARBON
    simulate = {}
    with model:
        simulate['default uptake'] = model.optimize().objective_value
    with model:
        model.medium = {i: 10.0 for i in base_medium}
        #print(model.medium)
        remove = []
        model.reactions.get_by_id('EX_o2_e').lower_bound = 0.
        # does not grow without oxygen
        carbon_sources.extend(NITROGEN)
        for source in carbon_sources:
            try:
                model.reactions.get_by_id(source).lower_bound = 0.
            except (KeyError):
                print(source + ' not in model')
                remove.append(source)
        print('Minimal medium to simulate on: ')
        print(model.medium)
        carbon_sources = [x for x in carbon_sources if x not in remove]
        simulate['no carbon'] = model.optimize().objective_value
        for source in carbon_sources:
            model.reactions.get_by_id(source).lower_bound = -10.
            solution = model.optimize().objective_value
            simulate[source] = solution#(np.log(2) / solution) * 60
            model.reactions.get_by_id(source).lower_bound = 0.
            
    return simulate

In [3]:
model = rg.io.load_model_cobra('../../models/Cstr_TS.xml')

### From default uptake

In [24]:
default_uptake = ['EX_amp_e', 'EX_arg__L_e', 'EX_ca2_e', 'EX_cl_e', 'EX_cobalt2_e',
       'EX_cu2_e', 'EX_dcyt_e', 'EX_fe3_e', 'EX_fol_e', 'EX_gly_asn__L_e',
       'EX_glyc3p_e', 'EX_glyglygln_e', 'EX_hishis_e', 'EX_istfrnA_e',
       'EX_k_e', 'EX_malthp_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_o2_e', 'EX_so4_e',
       'EX_tet_e', 'EX_thm_e', 'EX_udcpp_e', 'EX_ura_e', 'EX_zn2_e']


simulate_carbon_sources(model, default_uptake)

{'EX_amp_e': 10.0, 'EX_arg__L_e': 10.0, 'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cobalt2_e': 10.0, 'EX_cu2_e': 10.0, 'EX_dcyt_e': 10.0, 'EX_fe3_e': 10.0, 'EX_fol_e': 10.0, 'EX_gly_asn__L_e': 10.0, 'EX_glyc3p_e': 10.0, 'EX_glyglygln_e': 10.0, 'EX_hishis_e': 10.0, 'EX_istfrnA_e': 10.0, 'EX_k_e': 10.0, 'EX_malthp_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_o2_e': 10.0, 'EX_so4_e': 10.0, 'EX_tet_e': 10.0, 'EX_thm_e': 10.0, 'EX_udcpp_e': 10.0, 'EX_ura_e': 10.0, 'EX_zn2_e': 10.0}
EX_male_e not in model
EX_22bipy_e not in model
EX_boricacid_e not in model
EX_adocbl_e not in model
EX_pydam_e not in model
EX_nac_e not in model
EX_orn_e not in model
{'EX_amp_e': 10.0, 'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cobalt2_e': 10.0, 'EX_cu2_e': 10.0, 'EX_dcyt_e': 10.0, 'EX_fe3_e': 10.0, 'EX_fol_e': 10.0, 'EX_gly_asn__L_e': 10.0, 'EX_glyc3p_e': 10.0, 'EX_glyglygln_e': 10.0, 'EX_hishis_e': 10.0, 'EX_istfrnA_e': 10.0, 'EX_k_e': 10.0, 'EX_malthp_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_o2_e': 10

{'default uptake': 2.6682569403913177,
 'no carbon': 2.5752591694218725,
 'EX_gly_e': 2.6124169371659995,
 'EX_fum_e': 2.587443924076381,
 'EX_pyr_e': 2.5752591694218605,
 'EX_succ_e': 2.57525916942186,
 'EX_glc__D_e': 2.8971665655995937,
 'EX_urea_e': 2.6124169371660013,
 'EX_4abz_e': 2.5752591694218623,
 'EX_pnto__R_e': 2.5752989617954523,
 'EX_fru_e': 2.897166565599593,
 'EX_gam_e': 2.592646504536121,
 'EX_glyald_e': 2.7362128675107273,
 'EX_ala__L_e': 2.580543743146439,
 'EX_arg__L_e': 2.6681279484637734,
 'EX_cys__L_e': 2.5752591694219205,
 'EX_glu__L_e': 2.597418423664205,
 'EX_his__L_e': 2.575259169421864,
 'EX_leu__L_e': 2.575259169421861,
 'EX_lys__L_e': 2.6064509773005375,
 'EX_phe__L_e': 2.5900622167208844,
 'EX_pro__L_e': 2.590433958590245,
 'EX_ser__L_e': 2.6532359518092075,
 'EX_thr__L_e': 2.665712323143091,
 'EX_trp__L_e': 2.585113937260894,
 'EX_val__L_e': 2.575259169421861,
 'EX_cit_e': 2.5778508160488203,
 'EX_btn_e': 2.5752591694218605,
 'EX_ribflv_e': 2.575430181715

### From minimal uptake

In [35]:
col = '3'
minmed = pd.read_csv('../../analysis/minmal_media/Cstr_14_minmed_20220705.csv')
minmed = minmed[['Unnamed: 0', col]]
minmed = minmed[minmed[col] != 0]
minmed = minmed.set_index('Unnamed: 0').to_dict()[col]
minmed

{'EX_ca2_e': 0.004164,
 'EX_cl_e': 0.004164,
 'EX_cobalt2_e': 8e-05,
 'EX_cu2_e': 0.0005672,
 'EX_fe2_e': 0.0116184,
 'EX_glyc3p_e': 0.7888744000019106,
 'EX_h2o2_e': 778.3400322285635,
 'EX_k_e': 0.1561544,
 'EX_mg2_e': 0.00694,
 'EX_mn2_e': 0.0005528,
 'EX_ptrc_e': 76.65466605714116,
 'EX_so4_e': 0.2005008000001942,
 'EX_zn2_e': 0.0002728}

In [36]:
n = {k: minmed[k] for k in minmed.keys() & set(NITROGEN)} # test if nitrogen or carbon source is in minmed
n

{}

In [47]:
simulate_carbon_sources(model, minmed.keys())

EX_male_e not in model
EX_22bipy_e not in model
EX_boricacid_e not in model
EX_adocbl_e not in model
EX_pydam_e not in model
EX_nac_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
EX_orn_e not in model
Minimal medium to simulate on: 
{'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cobalt2_e': 10.0, 'EX_cu2_e': 10.0, 'EX_fe2_e': 10.0, 'EX_glyc3p_e': 10.0, 'EX_h2o2_e': 10.0, 'EX_k_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_ptrc_e': 10.0, 'EX_so4_e': 10.0, 'EX_zn2_e': 10.0}


{'default uptake': 52.6861223125543,
 'no carbon': 0.049380307335412724,
 'EX_gly_e': 0.05382664815057964,
 'EX_fum_e': 0.052953387346571776,
 'EX_pyr_e': 0.06057194491767355,
 'EX_succ_e': 0.05107009294273939,
 'EX_glc__D_e': 0.2899472037456511,
 'EX_urea_e': 0.04938030733522562,
 'EX_4abz_e': 0.04938349245882519,
 'EX_pnto__R_e': 0.049384119917016286,
 'EX_fru_e': 0.2899472037455379,
 'EX_gam_e': 0.0500296910486667,
 'EX_glyald_e': 0.17964158339732947,
 'EX_ala__L_e': 0.06562418085380968,
 'EX_arg__L_e': 0.07765715073389255,
 'EX_cys__L_e': 0.05202841181213044,
 'EX_glu__L_e': 0.0685758395065267,
 'EX_his__L_e': 0.05203067960182977,
 'EX_leu__L_e': 0.050043000556304375,
 'EX_lys__L_e': 0.057346294789401,
 'EX_phe__L_e': 0.050193737645107546,
 'EX_pro__L_e': 0.07548469070903784,
 'EX_ser__L_e': 0.08227358216821654,
 'EX_thr__L_e': 0.06678336395388716,
 'EX_trp__L_e': 0.04980570137753167,
 'EX_val__L_e': 0.05041677226459678,
 'EX_cit_e': 0.051457659956464896,
 'EX_btn_e': 0.04938030733

In [ ]:
ess = []
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        if model.objective.value <= 11:
            print('%s blocked (bounds: %s), new growth rate %f $' %
                  (reaction.id, str(reaction.bounds), model.objective.value))
            ess.append(reaction.id)
            
ess

## Minimal media


In [58]:
model = rg.io.load_model_cobra('../../models/Cstr_KC-Na-01.xml')
mini = minimal_medium(model, 0.1, minimize_components=4, open_exchanges=True)
mini.to_csv('/Users/baeuerle/Organisation/Masterarbeit/C_striatum_GEMs/analysis/minmal_media/Cstr_KC-Na-01_minmed_20230320.csv')
mini

,0,1,2,3
EX_ala_B_e,0.000000,0.000000,0.000056,0.000056
EX_ca2_e,0.000504,0.000504,0.000504,0.000504
EX_cgly_e,0.000000,0.000000,109.092671,0.000000
EX_cit_e,0.000504,0.000504,0.000504,0.000504
EX_cl_e,0.000504,0.000504,0.000504,0.000504
EX_cmp_e,0.032035,0.032035,0.032035,0.032035
EX_cobalt2_e,0.000010,0.000010,0.000010,0.000010
EX_cu2_e,0.000069,0.000000,0.000069,0.000069
EX_cu_e,0.000000,0.000069,0.000000,0.000000
EX_fe2_e,0.000000,0.001406,0.000000,0.001406


In [ ]:
from cobra.medium import minimal_medium
minimal_medium(model, 0.1, minimize_components=True)
#%%
min_medium = {'EX_ca2_e': 0.000521,
'EX_cl_e':             0.000521,
'EX_cobalt2_e':        0.000010,
'EX_cu2_e':         0.000071,
'EX_gly_cys__L_e':     3.929106,
'EX_istfrnB_e':     0.001452,
'EX_k_e':       0.019519,
'EX_mg2_e':        0.000868,
'EX_mn2_e':            0.000069,
'EX_o2_e':            10.000000,
'EX_pi_e':             0.098609,
'EX_so4_e':            0.000434,
'EX_zn2_e':            0.000034}
min_medium

import numpy as np
with model:
    model.medium = min_medium
    growth = model.slim_optimize() # = 0.099
    time = (np.log(2) / growth) * 60
time # 417

for key, value in min_medium.items():
    min_medium[key] = 10.0

with model:
    model.medium = min_medium
    growth = model.optimize() # = 0.099
    time = (np.log(2) / growth.objective_value) * 60
time # 390

In [ ]:
min2 = minimal_medium(model, 0.8, minimize_components=4, open_exchanges=True)
#%%
min2.to_csv('../analysis/growth/Cstr_14_minmed.csv')
#%%
aerob1 = min2[1].loc[min2[1] > 0] #aerob
aerob2 = min2[2].loc[min2[2] > 0] #aerob
min2[0].loc[min2[0] > 0] #anaerob
min2[3].loc[min2[3] > 0] #anaerob
#%%

a1_only = aerob1.reset_index()[~aerob1.reset_index()['index'].isin(aerob2.reset_index()['index'])].dropna()
a1_only = list(a1_only['index'])
['EX_chols_e', 'EX_ile__L_e', 'EX_istfrnA_e']
#%%
a2_only = aerob2.reset_index()[~aerob2.reset_index()['index'].isin(aerob1.reset_index()['index'])].dropna()
a2_only = list(a2_only['index'])
['EX_ala_L_Thr__L_e', 'EX_salchs4fe_e', 'EX_so4_e']
#%%
common = aerob2.reset_index()[aerob2.reset_index()['index'].isin(aerob1.reset_index()['index'])].dropna()
common=list(common['index'])
['EX_ca2_e',
 'EX_cl_e',
 'EX_cobalt2_e',
 'EX_cu2_e',
 'EX_k_e',
 'EX_mg2_e',
 'EX_mn2_e',
 'EX_o2_e',
 'EX_pi_e',
 'EX_zn2_e']

In [53]:
media = pd.read_csv('/Users/baeuerle/Organisation/Masterarbeit/refinegems/data/media_db.csv', sep=';')

M9 = media[media['medium']=='M9']
M9_ex = list('EX_' + M9['BiGG'] +'_e')

def remove_common(a, b):
  
    a, b = [i for i in a if i not in b], [j for j in b if j not in a]
  
    print("list1 : ", a)
    print("list2 : ", b)

print(a2_only)
remove_common(M9_ex, common)
remove_common(M9_ex, a1_only)
# for a2 there is missing ala_L_THr__L and salchs4fe in M9
# for a1 there is missing 'EX_chols_e', 'EX_ile__L_e', 'EX_istfrnA_e'

NameError: name 'a2_only' is not defined

In [56]:
#%% growthmedia testing
model = rg.io.load_model_cobra('../../models/Cstr_TS.xml')
minimal = pd.read_csv('/Users/baeuerle/Organisation/Masterarbeit/C_striatum_GEMs/analysis/minmal_media/Cstr_14_minmed_20220705.csv')
list(minimal['Unnamed: 0'])
#%%
M9 = media[media['medium']=='LB']
M9_ex = list('EX_' + M9['BiGG'] +'_e')
M9_ex.remove('EX_adn_e')
M9_ex.remove('EX_cbl1_e')
M9_ex.remove('EX_cd2_e')
M9_ex.remove('EX_cmp_e')
M9_ex.remove('EX_cro4_e')
M9_ex.remove('EX_dad_2_e')
M9_ex.remove('EX_gmp_e')
M9_ex.remove('EX_gsn_e')
M9_ex.remove('EX_hg2_e')
M9_ex.remove('EX_ins_e')
M9_ex.remove('EX_na1_e')
M9_ex.remove('EX_nac_e')
M9_ex.remove('EX_ni2_e')
M9_ex.remove('EX_pydx_e')
M9_ex.remove('EX_thymd_e')
M9_ex.remove('EX_ump_e')
M9_ex.remove('EX_uri_e')
print(M9_ex)
#M9_ex.remove('EX_glc__D_e')
with model:
    model.medium = {i:10 for i in M9_ex}
    sol = model.optimize().objective_value
    imp = []
    print(sol)
    min_ex = [i for i in list(minimal['Unnamed: 0']) if i not in M9_ex]
    for x in min_ex:
        model.reactions.get_by_id(x).lower_bound = - 10.0
        sol2 = model.optimize().objective_value
        if sol2 > sol:
            imp.append(x)
        model.reactions.get_by_id(x).lower_bound = 0
imp

['EX_ala__L_e', 'EX_amp_e', 'EX_arg__L_e', 'EX_aso3_e', 'EX_asp__L_e', 'EX_ca2_e', 'EX_cl_e', 'EX_cobalt2_e', 'EX_cu2_e', 'EX_cys__L_e', 'EX_dcyt_e', 'EX_fe2_e', 'EX_fe3_e', 'EX_fol_e', 'EX_glc__D_e', 'EX_glu__L_e', 'EX_gly_e', 'EX_h2o_e', 'EX_h2s_e', 'EX_h_e', 'EX_his__L_e', 'EX_hxan_e', 'EX_ile__L_e', 'EX_k_e', 'EX_leu__L_e', 'EX_lipoate_e', 'EX_lys__L_e', 'EX_met__L_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_mobd_e', 'EX_nh4_e', 'EX_o2_e', 'EX_phe__L_e', 'EX_pheme_e', 'EX_pi_e', 'EX_pnto__R_e', 'EX_pro__L_e', 'EX_ribflv_e', 'EX_ser__L_e', 'EX_so4_e', 'EX_thm_e', 'EX_thr__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_ura_e', 'EX_val__L_e', 'EX_zn2_e']
0.5208231015878155


['EX_glyc3p_e', 'EX_h2o2_e', 'EX_istfrnA_e', 'EX_ptrc_e', 'EX_salchs4fe_e']

In [57]:
def get_medium(media, name):
    RPMI_base = list('EX_' + media[media['medium']==name]['BiGG']+'_e')
    rpmi = []
    for i in RPMI_base:
        try:
            model.reactions.get_by_id(i)
            rpmi.append(i)
        except (KeyError):
            print(str(i) + ' is not in model')
    return rpmi

print(media)
medium = get_medium(media, 'SNM3')
with model:
    for reaction in model.reactions:
        if 'EX' in str(reaction.id[:2]):
            reaction.lower_bound = 0
    print(model.medium)
    model.medium = {i:10 for i in medium}
    print(model.medium)
    print(model.optimize())
    min_snm = minimal_medium(model, 0.1, minimize_components=True)
    minimal = list(pd.DataFrame(min_snm).reset_index()['index'])
    model.medium = {i:10 for i in minimal}
    print(model.medium)
    print(model.optimize())
    
#%%
# we have to look systematically at the flux distribution
m9 = get_medium(media, 'M9')
snm3 = get_medium(media, 'SNM3')
rpmi = get_medium(media, 'RPMI')

with model:
    model.medium = {i:10 for i in m9}
    print(model.medium)
    flux_m9 = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_m9'}, axis=1)

with model:
    model.medium = {i:10 for i in snm3}
    print(model.medium)
    flux_snm3 = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_snm3'}, axis=1)
    
with model:
    model.medium = {i:10 for i in rpmi}
    print(model.medium)
    flux_rpmi = pd.DataFrame(model.optimize().fluxes).rename({'fluxes':'fluxes_rpmi'}, axis=1)

#%%
all_flux = flux_rpmi.join(flux_m9).join(flux_snm3)
all_flux.loc[~(all_flux != 0).any(axis=1)] # 1578 reactions have no flux
all_flux.loc[(all_flux != 0).any(axis=1)] # only 428 have flux

#%%
model.optimize()
model.summary(fva=0.9)

     medium     description    BiGG        name
0        LB  Lysogeny broth     adn   Adenosine
1        LB             NaN  ala__L   L-Alanine
2        LB             NaN     amp         AMP
3        LB             NaN  arg__L  L-Arginine
4        LB             NaN    aso3   Arsenite 
..      ...             ...     ...         ...
351  CGXlab             NaN     h2o         NaN
352  CGXlab             NaN      o2         NaN
353  CGXlab             NaN     na1         NaN
354  CGXlab             NaN       h         NaN
355  CGXlab             NaN  34dhbz         NaN

[356 rows x 4 columns]
EX_orn_e is not in model
EX_male_e is not in model
nan is not in model
nan is not in model
EX_adocbl_e is not in model
EX_nac_e is not in model
EX_pydam_e is not in model
{}
{'EX_4abz_e': 10, 'EX_ala__L_e': 10, 'EX_arg__L_e': 10, 'EX_btn_e': 10, 'EX_ca2_e': 10, 'EX_cit_e': 10, 'EX_cl_e': 10, 'EX_co2_e': 10, 'EX_cobalt2_e': 10, 'EX_cu2_e': 10, 'EX_cys__L_e': 10, 'EX_fe2_e': 10, 'EX_fum_e': 10, 'EX_

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
4abut_e,EX_4abut_e,1000,[0; 1000],4,23.68%
6apa_e,EX_6apa_e,77.2,[0; 453.9],8,3.66%
acald_e,EX_acald_e,22.29,[-1000; 333.5],2,0.26%
ala_L_gln__L_e,EX_ala_L_gln__L_e,13.88,[0; 296.1],8,0.66%
amp_e,EX_amp_e,10.76,[0; 66.09],10,0.64%
arg__L_e,EX_arg__L_e,27.23,[-39.59; 1000],6,0.97%
asn__L_e,EX_asn__L_e,12.7,[0; 289.5],4,0.30%
ca2_e,EX_ca2_e,0.2742,[0.2468; 0.2742],0,0.00%
cl_e,EX_cl_e,0.2742,[0.2468; 0.2742],0,0.00%
cobalt2_e,EX_cobalt2_e,0.005269,[0.004742; 0.00516],0,0.00%


In [ ]:
RPMI_base = list('EX_' + media[media['medium']=='RPMI']['BiGG']+'_e')
RPMI = []
for i in RPMI_base:
    try:
        model.reactions.get_by_id(i)
        RPMI.append(i)
    except (KeyError):
        print(str(i) + ' is not in model')

with model:
    #RPMI.extend(common+a1_only)
    for reaction in model.reactions:
        if 'EX' in str(reaction.id[:2]):
            reaction.lower_bound = 0
    #model.medium = {i:10.0 for i in M9_working}
    for id in RPMI:#M9_working:
        model.reactions.get_by_id(id).lower_bound = -10
    model.reactions.get_by_id('EX_o2_e').lower_bound = -20
    #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -20
    print(model.medium)
    sol = model.optimize()
    print(sol.objective_value)
    fluxes = pd.DataFrame(sol.fluxes)
    fluxes = fluxes.loc[fluxes['fluxes'] != 0].reset_index()
    #print(fluxes.loc[fluxes['fluxes'] < 0].loc[fluxes['index'].apply(find_exchanges)])
    print(fluxes.loc[fluxes['fluxes'] > 0].loc[fluxes['index'].apply(find_exchanges)])
    # https://www.megazyme.com/l-lactic-acid-assay-kit
    ess = []
    rmpi = common + a1_only
    for ex in rmpi:
        model.reactions.get_by_id(ex).lower_bound = -10
        if model.slim_optimize() > 0:
            ess.append(ex)
        model.reactions.get_by_id(ex).lower_bound = 0
    print(ess+RPMI)
    model.medium = {i:10 for i in ess+RPMI}
    print(model.optimize())

